# AI Agent with Search and Weather tool capabilities.

In [4]:
from langchain_community.tools import tool, DuckDuckGoSearchResults
from langchain.agents import AgentExecutor, create_react_agent
from langchain import hub 
from langchain_openai import ChatOpenAI
import requests

## Creating ReAct prompt, llm model 

In [3]:
prompt = hub.pull("hwchase17/react")
llm = ChatOpenAI()

E:\Learning-Curve\Langchain\.venv\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


## Creating weather tool & DuckDuckGo search tool   

In [6]:
import os 
from dotenv import load_dotenv
load_dotenv() 
secret_key = os.environ['WEATHER_API_KEY']

In [15]:
search_tool = DuckDuckGoSearchResults()

@tool
def weather_tool(city: str) -> str:
    '''
        This function fetches current weather data for a given city. 
    '''
    try:
        url = f'http://api.weatherstack.com/current? access_key = ${secret_key}& query = ${city}'
        response = requests.get(url)
        return response.json()
    except:
        response = "Something went wrong."
        return response.json()

## Creating ReAct Agent manually

In [16]:
agent = create_react_agent(
    llm=llm,
    tools=[search_tool, weather_tool], 
    prompt=prompt
)

## Creating Agent Executor

In [17]:
agent_executor = AgentExecutor(
    tools=[search_tool, weather_tool],
    agent=agent, 
    verbose=True
)

In [18]:
response = agent_executor.invoke({"input":"What is the temperature of Satna and give me one speciality of Satna?"})
print(response)



> Entering new AgentExecutor chain...
I can use the weather_tool to find the temperature of Satna and then use Duck Duck Go to search for a speciality of Satna.
Action: weather_tool
Action Input: "Satna"{'success': False, 'error': {'code': 101, 'type': 'missing_access_key', 'info': 'You have not supplied an API Access Key. [Required format: access_key=YOUR_ACCESS_KEY]'}}It seems like I need an API Access Key to use the weather_tool. I will use Duck Duck Go to search for the weather in Satna instead.
Action: duckduckgo_results_json
Action Input: "weather in Satna"snippet: Satna - Weather forecast from Theweather.com. Weather conditions with updates on temperature, humidity, wind speed, snow, pressure, etc. for Satna, Madhya Pradesh, title: Satna Weather 14 days - Meteored, link: https://www.theweather.com/satna.htm, snippet: 12 day Satna Weather Forecast. Live Weather Warnings, hourly weather updates. Accurate Satna weather today, forecast for sun, rain, wind and temperature., title: 

In [21]:
print(response['output'])

The current temperature of Satna is 37°C and one speciality of Satna is its Jain temples and the Hindu Lakshminarayan Temple.
